In [2]:
'''
You can 1) perform smoothing on the original standard deviation 
or 2) calculate the standard deviation on the smoothing data.
If using 1), please use the smoothing in preprocess_data, 
If using 2), please use the smoothing in plot_mean;
To the best of our known, both are correct. Here we use the first method.
'''

import sys 
sys.path.append("/home/moon/data/exps/SFL/")

import os
import re
import numpy as np
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

from sim.utils.record_utils import read_fromcsv
from plots.plot_utils import moving_average, save_fig_timestamp

def preprocess_data(base_files, setup, path):
    seeds = [0,1,2]
    yi = setup['yi']

    data_settings = []
    # construct data with different seeds
    for i in range(len(base_files)):
        data_per_setting = [] # with multiple seeds
        data_last_xround_per_setting = [] # with multiple seeds
        for j in range(len(seeds)):
            tfile = re.sub(r'seed\d+', 'seed{}'.format(seeds[j]), base_files[i])
            tfile = f'{tfile}.csv' if '.csv' not in tfile else tfile
            #print(tfile)
            if not os.path.exists(os.path.join(path, tfile)):
                if setup['out']:
                    print('\tNot found {}'.format(tfile))
                    pass
                continue
            df = read_fromcsv(tfile, path)
            if df['round'].values[-1] != setup['end']:
                if setup['out']:
                    print('\tNot completed {}'.format(tfile))
                    pass
                continue
            # pick the data we want to plot
            # step = df['round'].values[1] - df['round'].values[0]
            # df = df[df['round'].isin([i for i in range(0, setup['end']+1, step)])]
            df = df[df['round'] <= setup['end']]
            # results over last x round 
            out = df.iloc[:, yi].values[-setup['select']:]
            data_last_xround_per_setting.extend(out.tolist())
            x = df['round'].values
            y = df.iloc[:, yi].values
            
            # if setup['smooth']:
            #     window_size = int(0.05*setup['end']*0.25) # 0.01*setup['end']*0.25
            #     y_smooth = moving_average(y_content, window_size)
            #     # adjust the length of the smooth data
            #     # discard `len(data)- (len(data)-len(windows)+1)=len(windows)-1` data points
            #     x_smooth = x[len(x)-len(y_smooth):]
            #     # back to the original variable
            #     x, y_content = x_smooth, y_smooth

            # data per seed
            if len(data_per_setting) == 0:
                data_per_setting = y
            else:
                data_per_setting =  np.vstack((data_per_setting, y))
        if setup['out']:
            pass
            print('Avg {}, {:5.2f}\\tiny{{$\\pm${:5<.2f}}}  {}'.format(len(data_last_xround_per_setting), np.mean(data_last_xround_per_setting), np.std(data_last_xround_per_setting), base_files[i]))
            #print('&{:5.2f}\\tiny{{$\\pm${:5<.2f}}}'.format(np.mean(data_last_xround_per_setting), np.std(data_last_xround_per_setting)), end=' ')
        data_settings.append(data_per_setting)
    return x, data_settings

    
def get_title(filename):
    args = filename.split('_')
    model_dict = {
        'logistic':  'Logisitc',
        'mlp': 'MLP',
        'lenet5': 'LeNet-5',
        'cnnmnist': 'CNN',
        'wvgg9k4': 'VGG-9',
        'resnetii18': 'ResNet-18'
        }
    #model = model_dict[args[5]]
    dataset_dict = {
        'mnist': 'MNIST', 
        'fashionmnist': 'FMNIST',
        'cifar10': 'CIFAR-10',
        'cifar100': 'CIFAR-100',
        'cinic10': 'CINIC-10'
    }
    dataset = dataset_dict[args[5]]
    #return '{}, {}, {}'.format(dataset, partition, K)
    return dataset

def format_y_ticks1(y, pos):
    return f'{y:.2f}'

def format_y_ticks2(y, pos):
    return f'{y:.0f}'

def plot_avg(ax, base_files, setup, path):    
    args = base_files[0].split('_')
    ylabels = ['Round', 'Training Loss', 'Training Top1 Accuracy (%)', 'Training Top5 Accuracy (%)', 'Test Loss', 'Test Top1 Accuracy (%)', 'Test Top5 Accuracy (%)']
    xlabel = 'Training rounds' 

    x, data_settings = preprocess_data(base_files, setup=setup, path=path)
    #print(len(np.array(data_settings).shape))
    for i, base_file in enumerate(base_files):
        if len(np.array(data_settings).shape) == 2:
            mean = data_settings[i]
            std = np.zeros_like(mean)
        else:
            mean = data_settings[i].mean(axis=0)
            std = data_settings[i].std(axis=0)
        # smooth
        if setup['smooth']:
            window_size = int(5) # 0.01*setup['end']*0.25
            mean_smooth = moving_average(mean, window_size)
            std_smooth = moving_average(std, window_size)
            x_smooth = x[len(x)-len(mean_smooth):]
            # back to the original variable
            x, mean, std = x_smooth, mean_smooth, std_smooth

        ax.plot(x, mean, linestyle=None, color='C{}'.format(i), lw=2, label=setup['label'][i])
        ax.fill_between(x, mean-std, mean+std, facecolor='C{}'.format(i), alpha=0.2)
        # plt.fill_between(x_axis, FedAvgdata[i].min(axis=0), FedAvgdata[i].max(axis=0), facecolor=colors[i], alpha=0.25) 
         
    #ax.set_title(get_title(base_file), fontsize=20)
    if setup['yi'] == 1 or setup['yi'] == 4:
        ax.set_yscale('log')
        #ax.set_ylim(ymax=1)
        pass
    if setup['yi'] == 1:
        ax.legend(loc='upper right', ncol=1, prop={'size': 18}) # loc = 1 or 4
    ax.set_ylabel(ylabels[setup['yi']], fontsize=16)
    ax.set_xlabel(xlabel, fontsize=16)
    ax.tick_params(labelsize=12)
    ax.set_xlim(xmin=0, xmax=setup['end'])
    
    #formatter = ticker.FuncFormatter(format_y_ticks2)
    #axs.yaxis.set_major_formatter(formatter)
    #axs.grid()
    #if 'exdir1' in patterns[0] and 'cifar10' in patterns[0] and 'wvgg9k4' in patterns[0]:
    #ax.legend(loc=4, ncol=1, prop={'size': 15}) # loc = 1 or 4
    #filename = 'mean_{}_{}_{}'.format(args[5], args[6], args[7])
    #print(filename)
    #plt.savefig('../figs/{}.png'.format(filename), bbox_inches='tight', dpi=300)


def main():
    dataset = 'fashionmnist'
    path = f'/home/moon/data/exps/SFL/save/{dataset}/'
    base_files = [
"FedAvg1.0_M1000,10_K5_R5000,10_wvgg9k4_cinic10_exdir5,100.0_sgd0.1,0.0,0.0_exp1.0_b20_seed0_clip10",
"CWT_M1000,10_K5_R5000,10_wvgg9k4_cinic10_exdir5,100.0_sgd0.01,0.0,0.0_exp1.0_b20_seed0_clip50",
    ]
    
    # fig, axs = plt.subplots(1, 4, figsize=(22, 4))
    # for i in range(4):
    #     plot_avg(axs[i], base_files, setup={'out': 1, "smooth": 1, 'yi': [1, 2, 4, 5][i], 'select': 10, 'end': 5000, 'label': ['PFL', 'SFL']}, path=path)
    # fig.tight_layout()
    #save_fig_timestamp(fig, '.png', '../figs/')

    # Create a global legend using the handles and labels from the first subplot
    #handles, labels = axs[0].get_legend_handles_labels()
    #fig.legend(handles, labels, loc='upper right', ncols=2)
    #fig.subplots_adjust(wspace=0.25)

    
#     setup = {'out': 1, "smooth": 1, 'yi': 5, 'select': 10, 'end': 5000, 'label': ['PFL', 'SFL']}
#     fig, axs = plt.subplots(1, 3, figsize=(13.5, 3.6))
#     base_files = [
# "FedAvg1.0_M500,10_K5_R5000,10_wvgg9k4_cifar10_exdir1,100.0_sgd0.1,0.0,0.0_exp1.0_b20_seed0_clip10",
# "CWT_M500,10_K5_R5000,10_wvgg9k4_cifar10_exdir1,100.0_sgd0.01,0.0,0.0_exp1.0_b20_seed0_clip50",
#     ]
#     plot_avg(axs[0], base_files, setup=setup, path=path)
    

#     base_files = [
# "FedAvg1.0_M500,10_K5_R5000,10_wvgg9k4_cifar10_exdir2,100.0_sgd0.1,0.0,0.0_exp1.0_b20_seed0_clip10",
# "CWT_M500,10_K5_R5000,10_wvgg9k4_cifar10_exdir2,100.0_sgd0.01,0.0,0.0_exp1.0_b20_seed0_clip50",

#     ]
#     plot_avg(axs[1], base_files, setup=setup, path=path)

#     base_files = [
# "FedAvg1.0_M500,10_K5_R5000,10_wvgg9k4_cifar10_exdir5,100.0_sgd0.1,0.0,0.0_exp1.0_b20_seed0_clip10",
# "CWT_M500,10_K5_R5000,10_wvgg9k4_cifar10_exdir5,100.0_sgd0.01,0.0,0.0_exp1.0_b20_seed0_clip50",
#     ]
#     plot_avg(axs[2], base_files, setup=setup, path=path)

#     for i in range(3):
#         axs[i].set_title('$C={}$'.format([1,2,5][i]), fontsize=18)
#         axs[i].set_ylim(ymin=20, ymax=85)
#     axs[0].legend(loc=4, ncol=1, prop={'size': 16}) # loc = 1 or 4
    
#     fig.tight_layout()


    #fig.savefig('{}/{}{}'.format(f'/home/moon/data/exps/sequential_local_sgd/figs/', 'test-acc-{}'.format(dataset), '.pdf'), bbox_inches='tight', dpi=300)
    base_files = [
"FedAvg1.0_M500,10_K5_R2000,10_cnn_fashionmnist_exdir1,100.0_sgd0.1,0.0,0.0_exp1.0_b20_seed0_clip10",
"CWT_M500,10_K5_R2000,10_cnn_fashionmnist_exdir1,100.0_sgd0.01,0.0,0.0_exp1.0_b20_seed0_clip50",

"FedAvg1.0_M500,10_K5_R2000,10_cnn_fashionmnist_exdir2,100.0_sgd0.1,0.0,0.0_exp1.0_b20_seed0_clip10",
"CWT_M500,10_K5_R2000,10_cnn_fashionmnist_exdir2,100.0_sgd0.01,0.0,0.0_exp1.0_b20_seed0_clip50",

"FedAvg1.0_M500,10_K5_R2000,10_cnn_fashionmnist_exdir5,100.0_sgd0.1,0.0,0.0_exp1.0_b20_seed0_clip10",
"CWT_M500,10_K5_R2000,10_cnn_fashionmnist_exdir5,100.0_sgd0.01,0.0,0.0_exp1.0_b20_seed0_clip50",
    ]
    preprocess_data(base_files, setup={'out': 1, "smooth": 1, 'yi': 2, 'select': 10, 'end': 2000, 'label': ['PFL', 'SFL']}, path=path)
    print('----')
    preprocess_data(base_files, setup={'out': 1, "smooth": 1, 'yi': 5, 'select': 10, 'end': 2000, 'label': ['PFL', 'SFL']}, path=path)
    #preprocess_data(base_files, setup={'out': 1, "smooth": 1, 'yi': 5, 'select': 10, 'end': 5000, 'label': ['PFL', 'SFL']}, path=path)
    #preprocess_data(base_files, setup={'out': 1, "smooth": 1, 'yi': 5, 'select': 10, 'end': 5000, 'label': ['PFL', 'SFL']}, path=path)
    #preprocess_data(base_files, setup={'out': 1, "smooth": 1, 'yi': 5, 'select': 10, 'end': 5000, 'label': ['PFL', 'SFL']}, path=path)

if __name__ == '__main__':
    main()

Avg 30, 86.71\tiny{$\pm$1.87}  FedAvg1.0_M500,10_K5_R2000,10_cnn_fashionmnist_exdir1,100.0_sgd0.1,0.0,0.0_exp1.0_b20_seed0_clip10
Avg 30, 88.86\tiny{$\pm$1.60}  CWT_M500,10_K5_R2000,10_cnn_fashionmnist_exdir1,100.0_sgd0.01,0.0,0.0_exp1.0_b20_seed0_clip50
Avg 30, 89.73\tiny{$\pm$1.23}  FedAvg1.0_M500,10_K5_R2000,10_cnn_fashionmnist_exdir2,100.0_sgd0.1,0.0,0.0_exp1.0_b20_seed0_clip10
Avg 30, 91.33\tiny{$\pm$1.49}  CWT_M500,10_K5_R2000,10_cnn_fashionmnist_exdir2,100.0_sgd0.01,0.0,0.0_exp1.0_b20_seed0_clip50
Avg 30, 92.27\tiny{$\pm$0.57}  FedAvg1.0_M500,10_K5_R2000,10_cnn_fashionmnist_exdir5,100.0_sgd0.1,0.0,0.0_exp1.0_b20_seed0_clip10
Avg 30, 92.83\tiny{$\pm$0.69}  CWT_M500,10_K5_R2000,10_cnn_fashionmnist_exdir5,100.0_sgd0.01,0.0,0.0_exp1.0_b20_seed0_clip50
----
Avg 30, 85.77\tiny{$\pm$1.96}  FedAvg1.0_M500,10_K5_R2000,10_cnn_fashionmnist_exdir1,100.0_sgd0.1,0.0,0.0_exp1.0_b20_seed0_clip10
Avg 30, 87.60\tiny{$\pm$1.56}  CWT_M500,10_K5_R2000,10_cnn_fashionmnist_exdir1,100.0_sgd0.01,0.0,0.0